In [49]:
import requests
from multiprocessing import Pool
import boto3

# set up db
db = boto3.resource('dynamodb', region_name = 'us-east-2')
table = db.Table('Inmates')

Plan
1. Use lambda to make calls in parallel? 
2. Store responses in Amazon Aurora

In [40]:
response = requests.get("https://www.bop.gov/PublicInfo/execute/inmateloc?todo=query&output=json&inmateNum=11111-001&inmateNumType=IRN")


In [65]:
params = {'todo': 'query', 'output': 'json', 'inmateNum': '69695-798', 'inmateNumType': 'IRN'}
response = requests.get("https://www.bop.gov/PublicInfo/execute/inmateloc", params)
response.json()

{'InmateLocator': [],
 'Captcha': False,
 'Messages': {},
 'FormToken': 'pub26740info'}

In [53]:
def sendGuess(params):
    inmateId = params['inmateId']
    regionId = params['regionId']
    
    inmateNum = '{0}-{1}'.format(str(inmateId)[1:], str(regionId)[1:])
    params = {
        'todo': 'query', 
        'output': 'json', 
        'inmateNum': inmateNum, 
        'inmateNumType': 'IRN'
    }
    response = requests.get("https://www.bop.gov/PublicInfo/execute/inmateloc", params)

    body = response.json()
    data = body['InmateLocator']
    if len(data):
        inmate = data[0]
        return inmate
    else: 
        return None
    

In [43]:
def transformApi(inmate):
    data = inmate
    data['region'] = inmate['inmateNum'].split('-')[-1]
    return data
            
    

In [44]:
def writeToTable(entry):
    with table.batch_writer() as batch:
        batch.put_item(Item=entry)

In [54]:
def handleGuess(params):
    response = sendGuess(params)
    if response is not None:
        payload = transformApi(response)
        writeToTable(payload)

In [59]:
def parallelGuessRange(params, process_count=4):
    inmateMin = params['inmateMin']
    inmateMax = params['inmateMax']
    regionNum = params['regionNum']
        
    # need to make a list which generates a search query
    batch = [
        {
            'inmateId': inmateId,
            'regionId': regionNum,
        } for inmateId in range(inmateMin, inmateMax)
    ]
    
    pool = Pool(processes = 4)
    pool.map(handleGuess, batch)

In [62]:
parallelGuessRange({
        'inmateMin': 100001,
        'inmateMax': 100100,
        'regionNum': 1000
    })

[{'inmateId': 100001, 'regionId': 1000}, {'inmateId': 100002, 'regionId': 1000}, {'inmateId': 100003, 'regionId': 1000}, {'inmateId': 100004, 'regionId': 1000}, {'inmateId': 100005, 'regionId': 1000}, {'inmateId': 100006, 'regionId': 1000}, {'inmateId': 100007, 'regionId': 1000}, {'inmateId': 100008, 'regionId': 1000}, {'inmateId': 100009, 'regionId': 1000}, {'inmateId': 100010, 'regionId': 1000}, {'inmateId': 100011, 'regionId': 1000}, {'inmateId': 100012, 'regionId': 1000}, {'inmateId': 100013, 'regionId': 1000}, {'inmateId': 100014, 'regionId': 1000}, {'inmateId': 100015, 'regionId': 1000}, {'inmateId': 100016, 'regionId': 1000}, {'inmateId': 100017, 'regionId': 1000}, {'inmateId': 100018, 'regionId': 1000}, {'inmateId': 100019, 'regionId': 1000}, {'inmateId': 100020, 'regionId': 1000}, {'inmateId': 100021, 'regionId': 1000}, {'inmateId': 100022, 'regionId': 1000}, {'inmateId': 100023, 'regionId': 1000}, {'inmateId': 100024, 'regionId': 1000}, {'inmateId': 100025, 'regionId': 1000},

In [64]:
mp.cpu_count()

2